In [2]:
import delf as df

In [3]:
import sys
import os.path as path

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)

from utiles import lectura_img
import numpy as np

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import numpy as np
from six.moves import range
import tensorflow as tf

from google.protobuf import text_format
from tensorflow.python.platform import app
from delf import delf_config_pb2
from delf import feature_io
from delf import utils
from delf import extractor

https://github.com/tensorflow/models/blob/master/research/delf/delf/python/examples/extract_features.py

# Original example

In [5]:
def main(image_paths, config_path, output_dir):
  # Read list of images.
  print('Reading list of images...')
  num_images = len(image_paths)
  print(f'done! Found {num_images} images')

  # Parse DelfConfig proto.
  config = delf_config_pb2.DelfConfig()
  with tf.io.gfile.GFile(config_path, 'r') as f:
    text_format.Merge(f.read(), config)

  # Create output directory if necessary.
  if not tf.io.gfile.exists(output_dir):
    tf.io.gfile.makedirs(output_dir)

  extractor_fn = extractor.MakeExtractor(config)

  start = time.time()
  for i in range(num_images):
    # Report progress once in a while.
    if i == 0:
      print('Starting to extract DELF features from images...')
    elif i % _STATUS_CHECK_ITERATIONS == 0:
      elapsed = (time.time() - start)
      print(
          f'Processing image {i} out of {num_images}, last '
          f'{_STATUS_CHECK_ITERATIONS} images took {elapsed} seconds'
      )
      start = time.time()

    # If descriptor already exists, skip its computation.
    out_desc_filename = os.path.splitext(os.path.basename(
        image_paths[i]))[0] + _DELF_EXT
    out_desc_fullpath = os.path.join(output_dir, out_desc_filename)
    if tf.io.gfile.exists(out_desc_fullpath):
      print(f'Skipping {image_paths[i]}')
      continue

    im = np.array(utils.RgbLoader(image_paths[i]))

    # Extract and save features.
    extracted_features = extractor_fn(im)
    locations_out = extracted_features['local_features']['locations']
    descriptors_out = extracted_features['local_features']['descriptors']
    feature_scales_out = extracted_features['local_features']['scales']
    attention_out = extracted_features['local_features']['attention']

    feature_io.WriteToFile(out_desc_fullpath, locations_out, feature_scales_out,
                           descriptors_out, attention_out)
    return descriptors_out

In [ ]:
# Extension of feature files.
_DELF_EXT = '.delf'
# Pace to report extraction log.
_STATUS_CHECK_ITERATIONS = 100

image_paths = lectura_img('D:\Archivos\Proyectos\minIA\images\GrupoDELF')
config_path = 'D:\Archivos\Proyectos\minIA\\notebooks\delf_config_example.pbtxt'
output_dir = 'D:\Archivos\Proyectos\minIA\descriptors\delf'
descrip = main(image_paths, config_path,output_dir)

## Modificación

In [6]:
import pickle
import cv2 as cv

In [7]:
# Pace to report extraction log.
_STATUS_CHECK_ITERATIONS = 10

def extractorDELF(image_dir, config_path, output_dir, name_pickle):
    
  # Read list of images.
    print('Reading list of images...')
    image_paths = lectura_img(image_dir)
    num_images = len(image_paths)
    print(f'Done! Found {num_images} images')

  # Parse DelfConfig proto.
    config = delf_config_pb2.DelfConfig()
    with tf.io.gfile.GFile(config_path, 'r') as f:
        text_format.Merge(f.read(), config)

  # Create output directory if necessary.
    if not tf.io.gfile.exists(output_dir):
        tf.io.gfile.makedirs(output_dir)
    
  #Create pickle file
    path_pickle = path.abspath(output_dir+'//'+name_pickle)
    pickle_file = open(path_pickle, 'wb')

  #Create lista de descriptor por imagen
    descriptors = list()
    
  #Crea el extractor
    extractor_fn = extractor.MakeExtractor(config)

    start = time.time()
    for i in range(num_images):
    # Report progress once in a while.
        if i == 0:
            print('Starting to extract DELF features from images...')
        elif i % _STATUS_CHECK_ITERATIONS == 0:
            elapsed = (time.time() - start)
            print(
                f'Processing image {i} out of {num_images}, last '
                f'{_STATUS_CHECK_ITERATIONS} images took {elapsed} seconds'
            )
            start = time.time()
            
      #Carga las imágenes
        im = np.array(utils.RgbLoader(image_paths[i]))

      # Extract features.
        extracted_features = extractor_fn(im)
        
      # Save features.
        nom_img = path.split(image_paths[i])[1]
        extracted_features['local_features']['name_img'] = nom_img
        descriptors.append(extracted_features['local_features'])
    pickle.dump(descriptors, pickle_file)
    print("Termino")
    return descriptors

In [16]:
#Output incluye el nombre del archivo
image_dir = 'D:\Archivos\Proyectos\minIA\images\GrupoDELF'
config_path = 'D:\Archivos\Proyectos\minIA\\notebooks\delf_config_example.pbtxt'
output_dir = 'D:\Archivos\Proyectos\minIA\descriptors\delf\PruebaD1'
name_pickle = 'pruebaDELF-08-10-20.pickle'
descrip = extractorDELF(image_dir, config_path, output_dir, name_pickle)

Reading list of images...
Done! Found 41 images
Starting to extract DELF features from images...
Processing image 10 out of 41, last 10 images took 11.350126266479492 seconds
Processing image 20 out of 41, last 10 images took 11.449545860290527 seconds
Processing image 30 out of 41, last 10 images took 10.341138124465942 seconds
Processing image 40 out of 41, last 10 images took 7.801088333129883 seconds
Termino


### Rutas de búsqueda de imágenes

In [17]:
print(f'Llaves de descriptores:\n {descrip[0].keys()} \n')

print('Descriptores encontrados por imagen:')
for i in range(len(descrip)):
    print(len(descrip[i]['descriptors']), end='-' )
    
print('\n\nTamaño de cada KeyPoint:')
for i in range(len(descrip)):
    print(len(descrip[i]['scales']), end='-' )
    
print('\n\nTamaño de cada descriptor imagen uno:')
for i in range(len(descrip[0]['descriptors'])):
    print(len(descrip[0]['descriptors'][i]), end='-' )
    
print('\n\nPuntuación de cada descriptr de la imagen uno')
print(descrip[0]['attention'], end='-' )

print(type(descrip[0]['descriptors']))

print(descrip[0]['locations'])

Llaves de descriptores:
 dict_keys(['locations', 'descriptors', 'scales', 'attention', 'name_img']) 

Descriptores encontrados por imagen:
20-21-19-26-24-20-20-28-23-34-28-13-19-40-17-14-49-7-77-26-20-41-28-32-22-44-31-35-23-37-21-21-8-11-29-25-193-207-140-284-266-

Tamaño de cada KeyPoint:
20-21-19-26-24-20-20-28-23-34-28-13-19-40-17-14-49-7-77-26-20-41-28-32-22-44-31-35-23-37-21-21-8-11-29-25-193-207-140-284-266-

Tamaño de cada descriptor imagen uno:
40-40-40-40-40-40-40-40-40-40-40-40-40-40-40-40-40-40-40-40-

Puntuación de cada descriptr de la imagen uno
[191.81944 173.71945 161.15355 153.3822  140.13965 139.81908 138.38896
 132.28749 124.10377 124.07345 122.19906 117.48806 110.6403  110.3928
 107.89728 105.55391 104.59865 103.08346 102.44146 101.6961 ]-<class 'numpy.ndarray'>
[[ 96.       256.      ]
 [ 90.510544 271.53162 ]
 [ 80.       256.      ]
 [ 90.51054  248.90399 ]
 [416.        64.      ]
 [ 96.       256.      ]
 [352.         0.      ]
 [  0.       288.      ]
 [192. 

### Creación de puntos de interés

In [22]:
#Recibe lista de keypoints de una imagen
def genKeyPoints( kp_img ):
    keypoints = list()
    for kp in kp_img:
        keypoints.append(cv.KeyPoint(kp[1], kp[0], 5))
    return keypoints

In [23]:
desc_muestras = descrip

for ima in desc_muestras:
        imagen = path.abspath(image_dir+'\\'+ima['name_img'])
        mini = (min(ima['locations'][0]), min(ima['locations'][1]) )
        maxi = (max(ima['locations'][0]), max(ima['locations'][1]))
        print(ima['name_img'], 'min:', mini,' max:', maxi)
        img = cv.imread(imagen)
        gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        keypoints = genKeyPoints(ima['locations'])
        img=cv.drawKeypoints(gray,keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv.imwrite(output_dir+'\\'+ima['name_img'],img)

100008.jpg min: (96.0, 90.510544)  max: (256.0, 271.53162)
100023.jpg min: (135.76581, 128.0)  max: (271.53162, 288.0)
100053.jpg min: (0.0, 112.0)  max: (181.02107, 128.0)
100078.jpg min: (22.627632, 22.627632)  max: (384.6698, 158.39345)
100090.jpg min: (0.0, 0.0)  max: (384.0, 384.6698)
100122.jpg min: (158.39345, 181.02109)  max: (362.04218, 203.64871)
100123.jpg min: (135.76581, 336.0)  max: (339.41452, 352.0)
100128.jpg min: (192.0, 32.0)  max: (384.0, 96.0)
100134.jpg min: (80.0, 203.64871)  max: (208.0, 248.90399)
100143.jpg min: (256.0, 128.0)  max: (256.0, 256.0)
100150.jpg min: (0.0, 158.39345)  max: (90.510544, 181.02109)
100157.jpg min: (160.0, 16.0)  max: (224.0, 256.0)
100187.jpg min: (48.0, 48.0)  max: (80.0, 144.0)
100204.jpg min: (224.0, 32.0)  max: (288.0, 80.0)
100237.jpg min: (22.627632, 22.627632)  max: (45.255272, 181.02109)
100259.jpg min: (32.0, 224.0)  max: (224.0, 384.0)
100263.jpg min: (90.51054, 176.0)  max: (248.90399, 352.0)
100288.jpg min: (226.27635, 13

skimage import feature